In [21]:
def GetStarting(latt = None, long = None, departement = None, type_request = None):
    
    if (type_request == "tgv_station"):
        return getTrainStationByPosition(latt, long, departement)
    
    else:
        return getUniversitiesByPosition(latt, long, departement)
     
        
def getTrainStationByPosition(latt = None, long = None, departement = None):
    
    g = rdflib.Graph()
    g.namespace_manager.bind('rdf', Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#'))
    g.namespace_manager.bind('rdfs', Namespace('http://www.w3.org/2000/01/rdf-schema#'))
    g.namespace_manager.bind('owl', Namespace('http://www.w3.org/2002/07/owl#'))
    g.namespace_manager.bind('tgv', Namespace('http://www.owl-ontologies.com/unnamed.owl#'))
    g.parse("data/gares-tgv/gares.owl")
    
    response = []
    euclid = []
    ToHtml = ""
    ToJs = ""
    
    if (departement == None):
        query = 'SELECT DISTINCT ?lat ?long ?nomGare  WHERE { ?gare tgv:Latitude ?lat . ?gare tgv:Longitude ?long . ?gare tgv:Nom_Gare ?nomGare }'
        res = g.query(query)

        for row in res:
            element =  row.asdict()
            response.append([element['nomGare'].toPython(),element['lat'].toPython(),element['long'].toPython()])
        for elt in response:
            euclid.append(sqrt( (latt - elt[1])**2 + (long - elt[2])**2))
        for i in range(len(euclid)):
            response[i].append(euclid[i])
        response = sorted(response, key=itemgetter(3))    
        
        response = response[:10]
        print(response)
        for i in range(len(response)):
            ToHtml += "<li><b>" + str(response[i][0]) + "</b> — lat: "  + str(response[i][1]) + "; lon: " + str(response[i][2]) + " — " + str(round(response[i][3]*111, 2)) + " km</li>"
            ToJs += "L.marker([" + str(response[i][1]) + ", " + str(response[i][2]) + "]).addTo(mymap).bindPopup('TGV station<br><b>" + str(response[i][0]) + "</b><br>lat. " + str(response[i][1]) + "; lon: " + str(response[i][2]) + "<br>" + str(round(response[i][3]*111, 2)) + " km away');\n"

    else:
        query = 'SELECT ?lat ?long ?nomGare  WHERE { ?gare tgv:Latitude ?lat . ?gare tgv:Longitude ?long . ?gare tgv:Nom_Gare ?nomGare . ?gare tgv:NOM_DEP ?nomDep FILTER regex(?nomDep, "^param") }'
        queryWithDepartement = query.replace("param",departement)
        res = g.query(queryWithDepartement)
        for row in res:
            element =  row.asdict()
            response.append([element['nomGare'].toPython(),element['lat'].toPython(),element['long'].toPython()])

        #print(response)
        for i in range(len(response)):
            ToHtml += "<li><b>" + str(response[i][0]) + "</b> — lat: "  + str(response[i][1]) + "; lon: " + str(response[i][2]) + "</li>"
            ToJs += "L.marker([" + str(response[i][1]) + ", " + str(response[i][2]) + "]).addTo(mymap).bindPopup('TGV station<br><b>" + str(response[i][0]) + "</b><br>lat. " + str(response[i][1]) + "; lon: " + str(response[i][2]) + "');\n"

    return (ToHtml, ToJs)


def getUniversitiesByPosition(latt = None, long = None, departement = None):
    
    g = rdflib.Graph()
    g.namespace_manager.bind('rdf', Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#'))
    g.namespace_manager.bind('rdfs', Namespace('http://www.w3.org/2000/01/rdf-schema#'))
    g.namespace_manager.bind('owl', Namespace('http://www.w3.org/2002/07/owl#'))
    g.namespace_manager.bind('IDF_Univ', Namespace('http://www.owl-ontologies.com/unnamed.owl#'))
    g.parse("data/universites/universites.owl")
    
    response = []
    euclid = []
    ToHtml = ""
    ToJs = ""
    
    if (departement == None):
        query = 'SELECT DISTINCT ?latitude ?longitude ?nom  WHERE { ?x IDF_Univ:latitude ?latitude . ?x IDF_Univ:longitude ?longitude . ?x IDF_Univ:nom ?nom }'
        res = g.query(query)

        for row in res:
            element =  row.asdict()
            response.append([element['nom'].toPython(),element['latitude'].toPython(),element['longitude'].toPython()])
        for elt in response:
            euclid.append(sqrt( (latt - elt[1])**2 + (long - elt[2])**2))
        for i in range(len(euclid)):
            response[i].append(euclid[i])
        response = sorted(response, key=itemgetter(3))    
        
        response = response[:8]
        print(response)
        for i in range(len(response)):
            ToHtml += "<li><b>" + str(response[i][0]) + "</b> — lat: "  + str(response[i][1]) + "; lon: " + str(response[i][2]) + " — " + str(round(response[i][3]*111, 2)) + " km</li>"
            ToJs += "L.marker([" + str(response[i][1]) + ", " + str(response[i][2]) + "]).addTo(mymap).bindPopup('University<br><b>" + str(response[i][0]) + "</b><br>lat. " + str(response[i][1]) + "; lon: " + str(response[i][2]) + "');\n"

    else:
        query = 'SELECT ?latitude ?longitude ?nom  WHERE { ?x IDF_Univ:latitude ?latitude . ?x IDF_Univ:longitude ?longitude . ?x IDF_Univ:nom ?nom . ?x IDF_Univ:dep ?dep FILTER regex(?dep, "^param") }'
        queryWithDepartement = query.replace("param",departement)
        res = g.query(queryWithDepartement)
        for row in res:
            element =  row.asdict()
            response.append([element['nom'].toPython(),element['latitude'].toPython(),element['longitude'].toPython()])

        response = response[:8]
        #print(response)
        for i in range(len(response)):
            ToHtml += "<li><b>" + str(response[i][0]) + "</b> — lat: "  + str(response[i][1]) + "; lon: " + str(response[i][2]) + "</li>"
            ToJs += "L.marker([" + str(response[i][1]) + ", " + str(response[i][2]) + "]).addTo(mymap).bindPopup('University<br><b>" + str(response[i][0]) + "</b><br>lat. " + str(response[i][1]) + "; lon: " + str(response[i][2]) + "');\n"

    return (ToHtml, ToJs)

## Api Flask

In [22]:
from flask import Flask, request, render_template, redirect, url_for, flash, jsonify

import pickle
import json

import rdflib
from math import *
from rdflib import Namespace
import sys
from operator import itemgetter

In [ ]:
app = Flask(__name__, template_folder='templates', static_url_path="/static", static_folder='static')

@app.route('/')
def home():
    return render_template("main.html")

@app.route('/result/', methods = ['POST'])
def predictSite():
    # Gathering input data
    if request.form['latitude'] == '':
        latitude = 46.30
    else:
        latitude = float(request.form['latitude'])
    if request.form['longitude'] == '':
        longitude = 2.30
    else:
        longitude = float(request.form['longitude'])
    if request.form['departement'] == '':
        departement = None
    else:
        departement = request.form['departement']
    if request.form['type_query'] == '':
        type_query = None
    else:
        type_request = request.form['type_query']
    data = [longitude, latitude, departement, type_request]
    type_text = ''
    if type_request == 'tgv_station':
        type_text = 'TGV stations'
    elif type_request == 'university':
        type_text = 'Universities'
    elif type_request == 'both':
        type_text = 'TGV stations and Universities'
    # Calculating by position
    output = GetStarting(latitude, longitude, departement, type_request)
    return render_template("result.html", output_text=output[0], output_map=output[1], latitude=latitude, longitude=longitude, type_text=type_text)

if __name__ == '__main__':
    app.run(debug = False, host='127.0.0.1')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2021 15:31:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2021 15:32:04] "POST /result/ HTTP/1.1" 200 -


[['Lycee polyvalent Saint Leon', 48.6092, 2.4826, 0.11057757458002178], ['Ecole departementale de theâtre', 48.6110594, 2.4778094, 0.11325463812453974], ["Institut catholique d'arts et metiers - site de Paris-Senart - ICAM ouvert", 48.613562040293, 2.5382333993912, 0.11982541393425422], ['Lycee Pierre Mendès France', 48.5965, 2.57648, 0.12313180092892252], ["CFA Faculte des metiers de l'Essonne - site d'Evry", 48.6226, 2.45731, 0.12981986019095698], ["Institut catholique d'arts et metiers - site de Paris-Senart - ICAM integre", 48.6252085, 2.5624680999999, 0.13992652353948962], ["Institut catholique d'arts et metiers - site de Paris-Senart - ICAM apprentissage", 48.631195, 2.551107, 0.14079791715078574], ["IUT d'Evry (site Evry - Romero - TCE)", 48.6230733, 2.4295124, 0.14182925976909824]]


127.0.0.1 - - [26/Mar/2021 15:32:10] "POST /result/ HTTP/1.1" 200 -


[['Massy TGV', 48.736, 2.291, 0.3152411775133424], ['Massy - Palaiseau', 48.731, 2.243, 0.3455575205374654], ['Paris-Montparnasse', 48.838, 2.317, 0.3843605078568824], ['Paris-Est', 48.871, 2.395, 0.38557230190977365], ['Paris-Nord', 48.881, 2.356, 0.40730455435705626], ['Versailles-Chantiers', 48.797, 2.136, 0.46979250739022876], ['Mantes-la-Jolie', 48.994, 1.721, 0.9224299431393149], ['Boulogne-Ville', 49.607, 2.707, 1.126187373397517], ['Amiens', 49.9, 2.289, 1.4158110749672768], ['Vendôme - Villiers-sur-Loir', 47.799, 1.036, 1.6231749751644153]]


...

127.0.0.1 - - [26/Mar/2021 15:34:05] "POST /result/ HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2021 15:34:16] "POST /result/ HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2021 15:34:42] "POST /result/ HTTP/1.1" 200 -


[['CFA CAMAS', 48.9673, 2.5717, 0.13568411845164474], ['Lycee Gustave Eiffel', 48.8892, 2.5422, 0.21858060298205695], ['Lycee Albert Schweitzer', 48.899, 2.51118, 0.21974984050051147], ['Lycee Madeleine Vionnet', 48.9043, 2.47433, 0.2325756627422587], ['Lycee Louise Michel', 48.9023, 2.44155, 0.25336079511242837], ['Lycee Jacques Brel', 48.9296, 2.40033, 0.26249622644906795], ['Ecole INFA', 48.8423, 2.48684, 0.2814506628167714], ['Lycee Paul Eluard', 48.9413, 2.36449, 0.283990581005781]]


127.0.0.1 - - [26/Mar/2021 15:36:04] "POST /result/ HTTP/1.1" 200 -


[['Institut de formation en soins infirmiers du Centre hospitalier du sud Seine-et-Marne', 48.4121, 2.69783, 0.3270735680240771], ['Lycee Jacques Amyot', 48.5462, 2.65958, 0.4501602119245966], ['Institut de formation en soins infirmiers du Centre hospitalier', 48.3836, 2.95065, 0.4509815766747017], ['Lycee polyvalent Leonard de Vinci', 48.5524, 2.65908, 0.4562413904941084], ['Lycee Pierre Mendès France', 48.5965, 2.57648, 0.497056777843334], ["Institut catholique d'arts et metiers - site de Paris-Senart - ICAM ouvert", 48.613562040293, 2.5382333993912, 0.5172630686417422], ['Lycee polyvalent Saint Leon', 48.6092, 2.4826, 0.5225585134700227], ['Ecole departementale de theâtre', 48.6110594, 2.4778094, 0.525463845622818]]


127.0.0.1 - - [26/Mar/2021 15:36:22] "POST /result/ HTTP/1.1" 200 -
